In [14]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [15]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
#clear_output()

Error: could not connect to ollama app, is it running?


In [16]:
!pip install -U lightrag[ollama]

In [17]:
!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time

from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

In [18]:
!pip install faiss-cpu


import re
import unicodedata
import pandas as pd

# Cargar datos del CSV
df = pd.read_csv('basecsvf.csv')

df['Respuesta'] = df['Respuesta'].apply(lambda x: str(x) if pd.notnull(x) else '')
df['Pregunta'] = df['Pregunta'].apply(lambda x: str(x) if pd.notnull(x) else '')

def normalize(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Pregunta'] = df['Pregunta']
df['Respuesta'] = df['Respuesta']

# Concatenar preguntas y respuestas
df['PreguntaRespuesta'] = df['Pregunta'] + " [SEP] " + df['Respuesta']

# Obtener la lista de textos concatenados
preguntas_respuestas = df['PreguntaRespuesta'].tolist()

from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'sentence-transformers/all-mpnet-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)


embeddings = get_embeddings(preguntas_respuestas)

import faiss
import numpy as np

d = 768

vectors = np.array([embedding.numpy() for embedding in embeddings], dtype='float32')

index = faiss.IndexFlatL2(d)

index.add(vectors)

faiss.write_index(index, 'mi_indice_faiss.index')

index = faiss.read_index('mi_indice_faiss.index')

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'sentence-transformers/all-mpnet-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


nueva_pregunta = "Hola, quién eres?"

nueva_embedding = get_embeddings([nueva_pregunta])[0].numpy().astype('float32')

distances, indices = index.search(np.array([nueva_embedding]), k=3)

similarity_percentage = 100 * (1 - distances[0][0])
print(f"Similitud: {similarity_percentage}%")

max_index = len(preguntas_respuestas) - 1
valid_indices = [i for i in indices[0] if i <= max_index]
respuestas_cercanas = [preguntas_respuestas[i] for i in valid_indices]

base_datos_vectorial = "\n".join(respuestas_cercanas)

print(base_datos_vectorial)
base_datos_vectorial = respuestas_cercanas
respuestas_cercanas

Similitud: -803.6664009094238%
¿En qué me puede ayudar bienestar estudiantil? [SEP] En el artículo 86 indica la s funciones de Unidad de Bienestar Estudiantil.- "Las instituciones de educación superior mantendrán una unidad administrativa de Bienestar Estudiantil destinada a promover la orientación vocacional y profesional, facilitar la obtención de créditos, estímulos, ayudas económicas y becas, y ofrecer los servicios asistenciales que se determinen en las normativas de cada institución. Esta unidad,
además, se encargará de promover un ambiente de respeto a los derechos y a la integridad física, psicológica y sexual de las y los estudiantes, en un ambiente libre de violencia, y brindará asistencia a quienes demanden por violaciones de estos derechos.
¿Por qué razón se me anularon mis materias? [SEP] 1. Por una mala gestión de las personas encargas de las aulas virtuales (Utic ) 
2 . No se cancelaron los pagos correspondientes a perdida de materias (en el caso que este realizando 2da 

['¿En qué me puede ayudar bienestar estudiantil? [SEP] En el artículo 86 indica la s funciones de Unidad de Bienestar Estudiantil.- "Las instituciones de educación superior mantendrán una unidad administrativa de Bienestar Estudiantil destinada a promover la orientación vocacional y profesional, facilitar la obtención de créditos, estímulos, ayudas económicas y becas, y ofrecer los servicios asistenciales que se determinen en las normativas de cada institución. Esta unidad,\nademás, se encargará de promover un ambiente de respeto a los derechos y a la integridad física, psicológica y sexual de las y los estudiantes, en un ambiente libre de violencia, y brindará asistencia a quienes demanden por violaciones de estos derechos.',
 '¿Por qué razón se me anularon mis materias? [SEP] 1. Por una mala gestión de las personas encargas de las aulas virtuales (Utic ) \n2 . No se cancelaron los pagos correspondientes a perdida de materias (en el caso que este realizando 2da o 3ra matricula) ',
 '¿

In [20]:
# Paso 5: Voy a Generar respuesta con el LLM
qa_template = r"""<SYS>
Eres una IA llamada ESPE Chatbot, respuestas SOLAMENTE relacionadas a la Universidad de las Fuerzas Armadas ESPE.
El usuario te va a hacer preguntas y tienes que responder en base a la siguiente base_datos_vectorial solo si está relacionado con la pregunta del user.
Si hay nombres, teléfonos, direciones o información relacionada en la base_datos_vectorial, da es información porque es segura.
La respuesta que otorgues debe ser natural, amigable y precisa, pero NO SALUDES.
Si no encuentras una respuesta adecuada en la base_datos_vectorial, proporciona una respuesta neutral sin mencionar base_datos_vectorial o pregunta de vuelta.
base_datos_vectorial (Escoge solo la respuesta más cercana a la pregunta):{{base_datos_vectorial}}
User: {{input_str}}
You:
<SYS>"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input), "base_datos_vectorial": base_datos_vectorial})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input), "base_datos_vectorial": base_datos_vectorial})

model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)
output = qa.call({"input_str": nueva_pregunta})
display(Markdown(f"**Answer:** {output}"))

**Answer:** GeneratorOutput(data='Soy ESPE Chatbot, una inteligencia artificial diseñada para brindar información y respuestas relacionadas con la Universidad de las Fuerzas Armadas ESPE. Estoy aquí para ayudarte con cualquier pregunta o tema relacionado con la universidad. ¿En qué puedo ayudarte hoy?', error=None, usage=None, raw_response='Soy ESPE Chatbot, una inteligencia artificial diseñada para brindar información y respuestas relacionadas con la Universidad de las Fuerzas Armadas ESPE. Estoy aquí para ayudarte con cualquier pregunta o tema relacionado con la universidad. ¿En qué puedo ayudarte hoy?', metadata=None)

In [21]:
class GeneratorOutput:
    def __init__(self, data=None, error=None, usage=None, raw_response=None, metadata=None):
        self.data = data
        self.error = error
        self.usage = usage
        self.raw_response = raw_response
        self.metadata = metadata

def extract_raw_response(output):
    if hasattr(output, 'raw_response') and output.raw_response is not None:
        return output.raw_response
    elif hasattr(output, 'data') and output.data is not None:
        return output.data
    return "Lo siento, no puedo procesar la respuesta en este momento."

clean_text = extract_raw_response(output)
print(clean_text)


Soy ESPE Chatbot, una inteligencia artificial diseñada para brindar información y respuestas relacionadas con la Universidad de las Fuerzas Armadas ESPE. Estoy aquí para ayudarte con cualquier pregunta o tema relacionado con la universidad. ¿En qué puedo ayudarte hoy?


# API EXPOSED


In [22]:
!pip install fastapi uvicorn nest-asyncio pyngrok


In [23]:
#FUNCIONA BIEN SOLO CON UNA PREGUNTA.
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Mantén la instancia de la app
app = FastAPI()

!ngrok config add-authtoken #token_ngrok
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Define el modelo de input para la API
class Query(BaseModel):
    question: str

# Aquí inicia la API para recibir la pregunta
@app.post("/ask")
async def ask_question(query: Query):
    nueva_pregunta = query.question

    # Genera el embedding de la nueva pregunta
    nueva_embedding = get_embeddings([nueva_pregunta])[0].numpy().astype('float32')

    # Busca en el índice de FAISS los embeddings más cercanos
    distances, indices = index.search(np.array([nueva_embedding]), k=3)

    # Filtra los resultados válidos
    max_index = len(preguntas_respuestas) - 1
    valid_indices = [i for i in indices[0] if i <= max_index]
    respuestas_cercanas = [preguntas_respuestas[i] for i in valid_indices]

    # Une las respuestas para enviarlas al modelo LLM
    base_datos_vectorial = "\n".join(respuestas_cercanas)

    # Genera la respuesta con el modelo LLM utilizando la plantilla y las respuestas cercanas
    output = qa.call({"input_str": nueva_pregunta, "base_datos_vectorial": base_datos_vectorial})

    # Limpia y extrae la respuesta generada
    clean_text = extract_raw_response(output)

    # Retorna la respuesta final
    return {"response": clean_text}

# Expose the API using ngrok
public_url = ngrok.connect(8000)
print(f"API running on {public_url}")

# Corre la aplicación FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
API running on NgrokTunnel: "https://c223-35-231-245-211.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [477]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [477]
